In [1]:
import pandas as pd
import pandahouse as ph

# Проект: вариант 2


## Задание 2. SQL

### 2.1 Очень усердные ученики.

Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".

Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин.

Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов.NB! Под усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.

In [2]:
# данные для подключения к БД default
connection_default = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

In [3]:
# создаем SQL запрос
sql_query_1 = '''
SELECT COUNT(st_id) as students_count
FROM (
    SELECT st_id
    FROM peas
    WHERE correct = 1
          AND EXTRACT(MONTH FROM timest) = (SELECT EXTRACT(MONTH FROM MAX(timest)) FROM peas)
    GROUP BY st_id
    HAVING COUNT(correct) >= 20
)
'''
# в данном запросе я беру количество студентов, из таблицы peas, где выбираю тех студентов, которые в текущем месяце решили 
# верно 20 и более задач. За текущий месяц я взял тот месяц, которому принадлежит последняя дата.

In [4]:
q_test_1 = ph.read_clickhouse(query=sql_query_1, connection=connection_default) # соединение с Clickhouse

In [5]:
diligent_student_counter = q_test_1.students_count[0] 

In [6]:
diligent_student_counter

136

#### Вывод: Количество очень усердных студентов, решивших 20 и более задач в текущем месяце(октябрь), равно 136.

### 2.2 Оптимизация воронки

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:

ARPU 
ARPAU 
CR в покупку 
СR активного пользователя в покупку 
CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике
ARPU считается относительно всех пользователей, попавших в группы.

Активным считается пользователь, за все время решивший больше 10 задач правильно в любых дисциплинах.

Активным по математике считается пользователь, за все время решивший 2 или больше задач правильно по математике.

### Создаем SQL запрос
#### Пояснения к запросу: 
    1. Пользователи - люди, которые зарегистрированы на платформе. 
    2. Клиенты - люди, которые совершили хотя бы 1 покупку

In [7]:

sql_query_2 = '''
SELECT test_grp,
ROUND(SUM(money) / COUNT(DISTINCT fpc.st_id), 2) as ARPPU,
ROUND(SUM(money) / (SELECT COUNT(DISTINCT st_id) FROM studs), 2) as ARPU,    
ROUND(COUNT(DISTINCT fpc.st_id) / (SELECT COUNT(DISTINCT st_id) FROM studs), 3) as cr_to_purchase,    
ROUND(COUNT(DISTINCT fpc.st_id) FILTER (WHERE  fpc.st_id IN (SELECT st_id
                                                             FROM peas  
                                                             WHERE correct = 1
                                                             GROUP BY st_id
                                                             HAVING COUNT(correct) > 10)) / (SELECT COUNT(DISTINCT st_id) FROM studs), 3) as cr_active,                                                                                                      
ROUND(COUNT(DISTINCT fpc.st_id) FILTER (WHERE subject = 'Math' and fpc.st_id IN (SELECT st_id
                                                                                 FROM peas  
                                                                                 WHERE correct = 1 and subject = 'Math'
                                                                                 GROUP BY st_id
                                                                                 HAVING COUNT(correct) >= 2)) / (SELECT COUNT(DISTINCT st_id) FROM studs), 4) as cr_active_math 
FROM final_project_check AS fpc 
INNER JOIN studs AS st ON fpc.st_id = st.st_id
GROUP BY test_grp                    
LIMIT 100
    '''


 #### В данном запросе,для каждой группы, мы считаем следующие метрики:
     1. ARPPU как отношение дохода, к числу клиентов для каждой группы
     2. ARPU как отношение дохода, к числу пользователей для каждой группы
     3. CR в покупку как отношение числа клиентов, к числу  пользователей для каждой группы
     4. CR активного пользователя в покупку как отношение числа активных пользователей, к числу пользователей для каждой группы
     5. CR активного пользователя в покупку как отношение числа активных по математике пользователей, к числу пользователей для каждой группы

In [8]:
q_test_2 = ph.read_clickhouse(query=sql_query_2, connection=connection_default)

In [9]:
q_test_2

,test_grp,ARPPU,ARPU,cr_to_purchase,cr_active,cr_active_math
0,control,92333.33,2308.33,0.025,0.023,0.0050
1,pilot,106093.75,5658.33,0.053,0.042,0.0067


#### Вывод: с помощью sql-запроса получены показатели необходимых метрик. На первый взгляд видно, что показатели группы pilot(тестовая группа) - выше. Можно провести стат.анализ и выяснить, являются ли изменения в показателях статистически значимыми.
    